In [185]:

import pandas as pd 
import numpy as np 


In [186]:
#dataset preparation 
a = pd.read_csv("../input/bank-marketing/bank-full.csv")



train = a.drop(['job','marital','education','contact', 'month', 'poutcome','default','balance','housing','loan','contact','month','duration','pdays','previous','poutcome'], axis=1)

# replace yes and no with 1 and 0
train['y'].replace({'no': 0, 'yes': 1},inplace = True)


X=train
Y=train['y']

# remove the label from X
X = X.drop(['y'], axis = 1)

In [187]:
X

,age,day,campaign
0,58,5,1
1,44,5,1
2,33,5,1
3,47,5,1
4,33,5,1
...,...,...,...
45206,51,17,3
45207,71,17,2
45208,72,17,5
45209,57,17,4


In [188]:
Y

0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int64

In [189]:
#activation function
def sigmoid(z):
    s = 1.0/ (1 + np.exp(-z))    
    return s

In [190]:
def network_architecture(X, Y):
    # nodes in input layer
    n_x = X.shape[0] 
    
    # nodes in hidden layer
    n_h = 10          
    # nodes in output layer
    n_y = Y.shape[0] 
    return (n_x, n_h, n_y)

In [191]:
def define_network_parameters(n_x, n_h, n_y):
    W1 = np.random.randn(n_h,n_x) * 0.01 # random initialization
    b1 = np.zeros((n_h, 1)) # zero initialization
    W2 = np.random.randn(n_h,n_h) * 0.01 # random initialization
    b2 = np.zeros((n_h, 1)) # zero initialization
    W3 = np.random.randn(n_y,n_h) * 0.01 
    b3 = np.zeros((n_y, 1)) 
    return {"W1": W1, "b1": b1, "W2": W2, "b2": b2,"W3": W3, "b3": b3}    

In [192]:
##Z = W*X + b
#A = g(Z)

#g is the activation function
#A is the activation using the input
#W is the weight associated with the input
#B is the bias associated with the node
def forward_propagation(X, params):
    Z1 = np.dot(params['W1'], X)+params['b1']
    A1 = sigmoid(Z1)

    Z2 = np.dot(params['W2'], A1)+params['b2']
    A2 = sigmoid(Z2)
    
    Z3 = np.dot(params['W3'], A2)+params['b3']
    A3 = sigmoid(Z3)
    return {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2,"Z3": Z3, "A3": A3}

In [193]:
def compute_error(Predicted, Actual):
    logprobs = np.multiply(np.log(Predicted), Actual)+ np.multiply(np.log(1-Predicted), 1-Actual)
    cost = -np.sum(logprobs) / Actual.shape[1] 
    return np.squeeze(cost)

In [194]:
def backward_propagation(params, activations, X, Y):
    m = X.shape[1]
    
    # output layer
    dZ3 = activations['A3'] - Y # compute the error derivative 
    dW3 = np.dot(dZ3, activations['A2'].T) / m # compute the weight derivative 
    db3 = np.sum(dZ3, axis=1, keepdims=True)/m # compute the bias derivative
    
    # hidden layer2
    dZ2 = np.dot(params['W3'].T, dZ3)*(1-np.power(activations['A2'], 2))
    dW2 = np.dot(dZ2,activations['A1'].T)/m
    db2 = np.sum(dZ2, axis=1,keepdims=True)/m
    
     # hidden layer1
    dZ1 = np.dot(params['W2'].T, dZ2)*(1-np.power(activations['A1'], 2))
    dW1 = np.dot(dZ1, X.T)/m
    db1 = np.sum(dZ1, axis=1,keepdims=True)/m
    
    
    
    return {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2 ,"dW3": dW3, "db3": db3}

def update_parameters(params, derivatives, alpha = 1.2):
    # alpha is the model's learning rate 
    
    params['W1'] = params['W1'] - alpha * derivatives['dW1']
    params['b1'] = params['b1'] - alpha * derivatives['db1']
    params['W2'] = params['W2'] - alpha * derivatives['dW2']
    params['b2'] = params['b2'] - alpha * derivatives['db2']
    params['W3'] = params['W3'] - alpha * derivatives['dW3']
    params['b3'] = params['b3'] - alpha * derivatives['db3']
    return params

In [195]:
def neural_network(X, Y, n_h, num_iterations=100):
    n_x = network_architecture(X, Y)[0]
    n_y = network_architecture(X, Y)[2]
    
    params = define_network_parameters(n_x, n_h, n_y)
    for i in range(0, num_iterations):
        results = forward_propagation(X, params)
        error = compute_error(results['A3'], Y)
        derivatives = backward_propagation(params, results, X, Y) 
        params = update_parameters(params, derivatives)    
    return params

In [196]:
y = Y.values.reshape(1, Y.size)
x = X.T.values
model = neural_network(x, y, n_h = 10, num_iterations = 10)

In [197]:
def predict(parameters, X):
    results = forward_propagation(X, parameters)
    print (results['A3'][0])
    predictions = np.around(results['A3'])    
    return predictions

predictions = predict(model, x)
print ('Accuracy: %d' % float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100) + '%')

[0.11702606 0.11726009 0.11750161 ... 0.11649172 0.11665651 0.11702206]
Accuracy: 88%
